In [2]:
#Title: Manafest pre-processing approach
#Author: Dusan Bosnjakovic / 03/05/2018
#Desc: This approach takes in a sample data set, generates data types for user confirmation of treatment
#    and lets the user specify whether variables are features/predictors or labels. Finally, the user can
#    specify what type of a lable is it (continuous, multi-class or binary). Intermediate files are saved
#    and the second stage can take in the sample files along with the manifest file.

import pandas as pd
import numpy as np
import qgrid
import sys
from sklearn.preprocessing import normalize
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import date
#import matplotlib.pyplot as plt

from sklearn.feature_selection import f_classif
from sklearn.preprocessing import MinMaxScaler#may not be used

############################
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor



############################
 
df = pd.read_excel('C:/Users/family/Pipeline/SHARE Data Set 2014 070218db_SM.xlsx', sheet_name='Refined Data3.0')
df_orig = df

projName='test2'
#with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    #display(hx)


In [3]:

df = df_orig
df=df[df['term']==' 36 months']
yearOfDataset = 2014


df['grade'] = np.where(df.grade == 'A', 1, 
              np.where(df.grade == "B", 2, 
              np.where(df.grade == "C", 3, 
              np.where(df.grade == "D", 4, 
              np.where(df.grade == "E", 5, 
              np.where(df.grade == "F", 6, 
              np.where(df.grade == "G", 7, -10)))))))

df['empLength'] = np.where(df.empLength == '10+ years', 15, 
              np.where(df.empLength == "1 year", 1, 
              np.where(df.empLength == "2 years", 2, 
              np.where(df.empLength == "3 years", 3, 
              np.where(df.empLength == "4 years", 4, 
              np.where(df.empLength == "5 years", 5,  
              np.where(df.empLength == "6 years", 6, 
              np.where(df.empLength == "7 years", 7, 
              np.where(df.empLength == "8 years", 8, 
              np.where(df.empLength == "9 years", 9,
              np.where(df.empLength == "< 1 year", 0, -10)))))))))))

df['bcutil'].fillna(-1000, inplace=True)
df['bcOpenToBuy'].fillna(-1000, inplace=True)
df['mthsSinceLastDelinq'].fillna(3000, inplace=True)
df['mthsSinceLastRecord'].fillna(3000, inplace=True)
df['mthssincelastmajorderog'].fillna(3000, inplace=True)
df['moSinOldIlAcct'].fillna(3000, inplace=True)
df['mthsSinceRecentBc'].fillna(3000, inplace=True)
df['mthsSinceRecentBcDlq'].fillna(3000, inplace=True)
df['mthsSinceRecentRevolDelinq'].fillna(3000, inplace=True)
df['numTl120dpd2m'].fillna(0, inplace=True)## prob impute median
df['percentBcGt75'].fillna(-100, inplace=True)## prob impute median
df['revolUtil'].fillna(-100, inplace=True)## prob impute median


df['FreeCashFlow'] = np.where(df.annualInc == 0, 1*(df.annualInc/12)-((df.annualInc*df.dti)/1200),
              np.where(df.annualInc <=25000, 0.97*(df.annualInc/12)-((df.annualInc*df.dti)/1200), 
              np.where(df.annualInc <50000, 0.92*(df.annualInc/12)-((df.annualInc*df.dti)/1200), 
              np.where(df.annualInc <100000, 0.88*(df.annualInc/12)-((df.annualInc*df.dti)/1200), 
              np.where(df.annualInc <200000, 0.82*(df.annualInc/12)-((df.annualInc*df.dti)/1200), 
              np.where(df.annualInc <500000, 0.72*(df.annualInc/12)-((df.annualInc*df.dti)/1200), 
              np.where(df.annualInc >=500000, 0.69*(df.annualInc/12)-((df.annualInc*df.dti)/1200),-10000)))))))

df['InstlToFCF'] = df.installment/df.FreeCashFlow


df['earliestCrLine_alt'] = yearOfDataset-pd.to_datetime(df['earliestCrLine']).dt.year

#impute
#bcOpenToBuy
#bcutil


C:\Users\family\Anaconda3\envs\LC\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\family\Anaconda3\envs\LC\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\family\Anaconda3\envs\LC\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

In [4]:
#config

maximumMissingness=0.2 #usually put to 0.2
minimumSampleForFeatureSelection=300000 #good number is 50000 for performance, but depends on baseline/n/m
minimumVarianceThreshold=0# default is 0.001

featureSelectionRTreeN=70
cv=False#set to true for more thorough feature evaluation
testProp =0.25

In [14]:
hx=pd.DataFrame(data=df.dtypes, index=None,columns=['d'])#,columns=['var','dTypes']
#hx.dtypes
d={'types':df.dtypes.astype('str'),'vars':hx.index.values.astype('str')}
e=pd.DataFrame(d)
#e['w']= np.where(e['types']=='float64', 'yes', 'no')
#e['ee']= 4
#e["types2"] = e["types"].astype('category')

#print(e.dtypes)
#print(e["types2"].cat.categories.tolist())
#print(e.types.unique())
###################
catVals=e[(e['types']=='category') | (e['types']=='object')].index.values
distinctVals = pd.DataFrame(df[catVals].T.apply(lambda x: x.nunique(), axis=1)).rename(columns={0: 'distinctVals'})
#print(distinctVals)
#manifest = manifest.merge(distinctVals, left_index=True,right_index=True, how='left')
e['distinctVals']=distinctVals

###################


def treatmentRFx(row):
    if row['types'] == 'object':
        if row['distinctVals'] >1000:
            return 'text'
        elif row['distinctVals'] ==1:
            return 'Ignore'
        else:
            return 'categorical'
    elif row['types'] =='float64':
        return 'numeric' 
    elif row['types'] =='datetime64[ns]':
        return 'Ignore' 
    elif row['types'] =='int64':
        return 'numeric' 
    elif row['types'] =='int32':
        return 'numeric' 
    elif row['types'] =='category':
        return 'categorical' 
    else:
        return 'Ignore'

def classRFx(row):
    if (row['types'] == 'datetime64[ns]') | (row['treatment'] == 'text'):
        return 'Ignore'
    elif row['vars'] in (['label'],['target'],['response']):
        return 'IDColumnIgnore'
    elif 'ID' in row['vars']:
        return 'IDColumnIgnore'
    else:
        
        return 'Predictor'#Predictor
    
treatment = e.apply(treatmentRFx, axis=1).astype('category')
treatment.cat.set_categories(['text','numeric','Ignore','categorical'],inplace=True)
e["treatment"] =treatment 

#class = e.apply(classRFx, axis=1).astype('category')
e['class'] = e.apply(classRFx, axis=1).astype('category')
#e['class']='Predictor'
#e['class']=e['class'].astype('category')
#classes2 = pd.Series(pd.Categorical(['Predictor']))

#type2=
e['class'].cat.set_categories(['IDColumnIgnore','Ignore','Predictor','LabelRegress','LabelBinary','LabelMultiClass'],inplace=True)
#print(classes)


e=e.drop(['vars'], axis=1)

#e.w=2
#print(1 if true else 0)

inputSetup=qgrid.show_grid(e,
                grid_options={'fullWidthRows': True,
                              'syncColumnCellResize': True,
                              'forceFitColumns': True,
                              'rowHeight': 40,
                              'enableColumnReorder': True,
                              'enableTextSelectionOnCells': True,
                              'editable': True})

#widget=qgrid.show_grid(e)
print('NOTE: only treatment and class columns changes will be saved')
inputSetup

#widget.get_changed_df()
#type(hx.dtypes)
#hx.dtypes.values
#hz=pd.DataFrame(data=hx.index.values, index=None,columns=['x'])
#h2=pd.concat([hz,hx],axis=1)
#hz
#hx




NOTE: only treatment and class columns changes will be saved


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [15]:
manifest=inputSetup.get_changed_df()
manifest=manifest.drop(['distinctVals'], axis=1)



#die;

if len(manifest[manifest['class']=='Predictor'])==0:
    print('ERROR: Please select at least 1 predictor in the "class" column')
elif len([manifest.iloc[i,j] for i,j in zip(*np.where(manifest.applymap(lambda x: x == '')))])>0:
    print('ERROR: There are missing values in the manifest')
elif len([manifest.iloc[i,j] for i,j in zip(*np.where(pd.isnull(manifest)))])>0:
    print('ERROR: There are NaN values in the manifest')
    

if len(manifest[manifest['class'].str.contains("Label")])==0:
    print('ERROR: Please select 1 label type in the "class" column')
elif len(manifest[manifest['class'].str.contains("Label")])>1:
    print('ERROR: You have selected more than one label type in the "class" column. Only 1 is allowed')
elif len(manifest[manifest['class'].str.contains("LabelRegress")])>0:
    regVar = manifest[manifest['class'].str.contains("LabelRegress")].index.values
    print("You have chosen a regression model to predict: "+str(regVar))
    regressVar = manifest[manifest['class'].str.contains("LabelRegress")].index.values
    dtype=manifest[manifest['class'].str.contains("LabelRegress")]['types']
    dtype=dtype[0]
    if(dtype!='float64' and dtype!='int64'):
        print('ERROR: You have selected a regression label but the label values aren\'t not numeric, but rather '+str(dtype))
elif len(manifest[manifest['class'].str.contains("LabelBinary")])>0:
    binVar = manifest[manifest['class'].str.contains("LabelBinary")].index.values
    print("You have chosen a binary classification model to predict: "+str(binVar))
    if df[binVar].nunique()[0]!=2:
        print('ERROR: You have selected a binary label but the number of unique values for that label not 2, but rather '+str(df[binVar].nunique()[0]))




print('Data frame length is: '+str(len(df)))
print('You have selected '+str(len(manifest[manifest['class']=='Predictor']))+' predictors')
if len(manifest[manifest['class'].str.contains("LabelMultiClass")])>0:
    mcVar = manifest[manifest['class'].str.contains("LabelMultiClass")].index.values
    print("You have chosen a multiclass classification model to predict: "+str(mcVar))
    multiLabelVar = manifest[manifest['class'].str.contains("LabelMultiClass")].index.values
    if df[multiLabelVar].nunique()[0]>100:
        print('NOTICE: You have selected a multiclass label with high number of unique values: '+str(df[multiLabelVar].nunique()[0]))
    else:
        print('Your multiclass label has this many unique values: '+str(df[multiLabelVar].nunique()[0]))
    
        

#np.where(pd.isnull(df))    


#np.where(manifest.applymap(lambda x: x == ''))

You have chosen a binary classification model to predict: ['DietzBool']
Data frame length is: 9593
You have selected 71 predictors


In [16]:

import pickle
#favorite_color = { "lion": "yellow", "kitty": "red" }
pickle.dump( manifest, open( projName+"_manifest.p", "wb" ) )

In [17]:
from sklearn.model_selection import train_test_split


missingTestSet=manifest[(manifest['class']=='Predictor') | (manifest['class'].isin(['Predictor','LabelBinary','LabelMultiClass','LabelRegress']))]
missingTestSet.index.values
reducedDF=df[missingTestSet.index.values]
missingStats=round((len(reducedDF.index)-reducedDF.count())/len(df),3)


labCol = manifest[manifest['class'].isin(["LabelRegress",'LabelBinary',"LabelMultiClass"])].index.values
labType = manifest[manifest['class'].isin(["LabelRegress",'LabelBinary',"LabelMultiClass"])]['class']

print('Text columns will not be selected for this evaluation.')

#die
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))
######################
#univariate stats
if (len(manifest[manifest['class'].isin(["LabelRegress",'LabelBinary',"LabelMultiClass"])]))==1:
    nonMissingDF = df[missingStats[missingStats<20].index.values]
    ValidPredictors=manifest[(manifest['class']=='Predictor') & ((manifest['treatment']=='numeric') | (manifest['treatment']=='categorical'))]
    XVars = intersection(ValidPredictors.index.values,missingStats[missingStats<maximumMissingness].index.values)#intersects numeric predictors and non missing values
    #print(intersection(XVars.index.values,missingStats[missingStats<20].index.values))
    #die;
    uvDF=nonMissingDF[XVars]
    uvDF2 = uvDF.dropna()
    print("dropped "+str(len(uvDF)-len(uvDF2))+" rows due to nas")
    

    
    
    #uvDF2=uvDF2.sample(n=min(minimumSampleForFeatureSelection,len(uvDF2)))

    #type(uvDF2)
    X = uvDF2.values
   
    Y=df[labCol].values
    Z = pd.concat([uvDF2, df[labCol]], axis=1, join='inner')
    Z = Z.dropna()
    
    X = pd.get_dummies(Z[XVars])
    Y = Z[labCol]
    
    
    ############################################################################
    ################### text treatment start
    ############################################################################
    from sklearn.feature_extraction.text import TfidfVectorizer
    import re

    def tokenizer(text):
        if text:
            result = re.findall('[a-z]{2,}', text.lower())
        else:
            result = []
        return result

    vect = TfidfVectorizer(tokenizer=tokenizer, stop_words='english',token_pattern=r'\w{2,}'
                           ,ngram_range=(1, 2)
                           ,max_df=0.5
                           , min_df=.0005,max_features=100
                          )

    TextPreds=manifest[(manifest['class']=='Predictor') & (manifest['treatment']=='text')].index.values
    reducedTextDF=df[TextPreds]
    #reducedTextDF=reducedTextDF.iloc[Z.uvDF2]
    
    textFeatures = pd.DataFrame()
    for i in range(0, len(TextPreds)):

        #i=0

        reducedSingleTextDF = reducedTextDF[TextPreds[i]].str.replace('\d+', '')#remove numbers
        reducedSingleTextDF.fillna(value='SpecialTextMissingValue', inplace=True)

        X1 = (reducedSingleTextDF).values

        X_train_vect = vect.fit_transform(X1)

        #X_train_vect.to_dense()
        X_train_vectSDF = pd.SparseDataFrame(X_train_vect)#.fillna(0, inplace=True)
        X_train_vectSDF = X_train_vectSDF.fillna(0)

        cols = [TextPreds[i]+'_'+str(s) for s in list(X_train_vectSDF)]

        X_train_vectDF = pd.DataFrame(X_train_vect.todense(),columns=cols,index=reducedTextDF.index)

        textFeatures = pd.concat([textFeatures, X_train_vectDF], axis=1, sort=False)

    ############################################################################
    ################### text treatment end
    ############################################################################
    

    

    
    
    
    #print(textFeatures.isnull().values.any())Z = pd.concat([uvDF2, df[labCol]], axis=1, join='inner')
    print(len(textFeatures))
    print(len(X))
    print(len(Y))
    #print(textFeatures.head())
    #print(X.head())
    #print(Y.head())
    
    if(len(textFeatures)>0):
        X = pd.concat([textFeatures, X], axis=1, sort=False, join='inner')
    #print(len(pd.concat([textFeatures, X], axis=1, sort=False, join='inner')))
    
    #X=X.append(textFeatures)
    #print(X.isnull().values.any())
    #print(X.shape)
    print(len(X))
    

    colNames = list(X)
    
    
    #print(colNames)
    
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = pd.DataFrame(scaler.fit_transform(X),columns=colNames)
    
    
    #result = df1.append(df2)

    
    
    #########################
    
    from sklearn.feature_selection import VarianceThreshold
    def variance_threshold_selector(data, threshold=0):
        selector = VarianceThreshold(threshold)
        selector.fit(data)
        return data[data.columns[selector.get_support(indices=True)]]

    vts=variance_threshold_selector(X,threshold=minimumVarianceThreshold)
    X = X[list(vts)]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, Y, test_size=testProp, random_state=None)
    

    #list(zip(list(vts)))
    #list(vts)
    #len(list(X[list(vts)]))
    #len(list(X))
    #print(list(X[list(vts)]))
    
    #Low var test
    
    #numericDF = df[manifest[(manifest['class']=='Predictor') & (manifest['treatment']=='numeric')].index.values]
    #sel = VarianceThreshold(threshold=(.5 * (1 - .5)))
    #sel.fit_transform(X)
    #numericDF.head()
    

    
    # Create the random grid

    
#{'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4
#, 'max_features': 'auto', 'max_depth': None, 'bootstrap': True}

    #print(list(X))
    if(labType.values=="LabelRegress"):
        print('regressing...')
        
        
        if(cv==True):
            from sklearn.model_selection import RandomizedSearchCV
                # Number of trees in random forest
            n_estimators = [int(x) for x in np.linspace(start = 200, stop = 400, num = 3)]
            # Number of features to consider at every split
            max_features = ['sqrt']
            # Maximum number of levels in tree
            max_depth = [3,5]#[int(x) for x in np.linspace(2, 5, num = 3)]
            max_depth.append(None)
            # Minimum number of samples required to split a node
            min_samples_split = [4, 5, 6]
            # Minimum number of samples required at each leaf node
            min_samples_leaf = [3, 4,5]
            # Method of selecting samples for training each tree
            bootstrap = [False]
            
            random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
            rf = RandomForestRegressor()
            cvfit = RandomizedSearchCV(estimator = rf, param_distributions = random_grid
                                 , n_iter = 15, cv = 7, verbose=2, random_state=None, n_jobs = -1)
            cvfit.fit(X_train, y_train.values.ravel())
            print(cvfit.best_params_)
            clf = cvfit.best_estimator_
            
        else:
            clf = RandomForestRegressor(max_depth=5,random_state=None,n_estimators=30,min_samples_leaf=4
                                        ,min_samples_split= 5,max_features='auto',bootstrap= True)


        clf.fit(X_train, y_train.values.ravel())
        #print(clf.best_params_)
                                       
        #clf.fit(X, Y.values.ravel())
        from sklearn.metrics import mean_squared_error
        from math import sqrt
        rms = sqrt(mean_squared_error(y_test.values.ravel(), clf.predict(X_test)))
        print("RMSQ: "+str(round(rms,3)))
    else:
        print('classifying...')
        clf = RandomForestClassifier(max_depth=2, random_state=0,n_estimators=featureSelectionRTreeN)
        clf.fit(X, Y.values.ravel())
        if(labType.values=="LabelBinary"):
            from sklearn.metrics import roc_auc_score
            auc = roc_auc_score(Y.values.ravel(), clf.predict(X))
            print("AUC: "+str(round(auc,3)))
        elif(labType.values=="LabelMultiClass"):
            from sklearn.metrics import accuracy_score
            #lrap = label_ranking_average_precision_score(Y.values.ravel(), clf.predict(X))
            acc=accuracy_score(Y.values.ravel(), clf.predict(X))
            print("Accuracy: "+str(round(acc,3)))
        
    
    #print(zip(clf.feature_importances_,list(X)))
    #print(pd.concat(list(X), clf.feature_importances_, axis=1, join='inner'))
    #print((X[['home_ownership']]))
    #print(pd.get_dummies(X[['home_ownership','installment']]))
    
    bestModel = clf
    
    

Text columns will not be selected for this evaluation.
dropped 1 rows due to nas
0
9592
9592
9592
classifying...
AUC: 0.968


In [18]:
import pickle
#favorite_color = { "lion": "yellow", "kitty": "red" }
pickle.dump( bestModel, open( projName+"_bestModelForFeatureEng.p", "wb" ) )


In [19]:
#import pickle
#favorite_color = { "lion": "yellow", "kitty": "red" }
#pickle.dump( optimized_GBM.best_estimator_, open( "xgb.p", "wb" ) )

In [20]:
#print(list(X))

#bestModel

#print(clf.feature_importances_)
#clf.best_estimator_
#print(list(zip(list(X), clf.feature_importances_)))
varImp=pd.DataFrame(list(zip(list(X), bestModel.feature_importances_)),columns=('vars','featureImpVal'))
varImp.sort_values(by=['featureImpVal'], ascending=False)

#print(manifest[manifest['treatment']=='categorical'])
#varImp[varImp['vars'].str.contains("emp_title"+"_")]
#cats=manifest[manifest['treatment']=='categorical'].index
#print(list(Z[XVars]))
#print(varImp[varImp['vars'].isin(list(Z[XVars]))])

#for index,row in varImp.iterrows():
    #if  row['vars'] in manifest[manifest['treatment']=='categorical'].index.values+"_":
    #print(1)

featureImp = pd.DataFrame(varImp[varImp['vars'].isin(list(Z[XVars]))])


for index,row in manifest[((manifest['treatment']=='categorical') | (manifest['treatment']=='text')) & (manifest['class']=='Predictor')].iterrows():
    #print(varImp[varImp['vars'].str.contains(str(row.name)+"_")]['featureImpVal'].mean())
    #featureImp[row.name][varImp[varImp['vars'].str.contains(str(row.name)+"_")]['featureImpVal'].mean()]
    featureImp = featureImp.append({'vars': row.name,'featureImpVal': round(varImp[varImp['vars'].str.contains(str(row.name)+"_")]['featureImpVal'].mean(),7)}, ignore_index=True)
    #print(str(row.name))
#print(featureImp)
   
featureImp=featureImp.dropna()
featureImp=featureImp.set_index('vars')

#print(manifest[manifest['treatment']=='categorical']].name)
#print(featureImp.sort_values(by=['featureImpVal'], ascending=False))
#print(pd.DataFrame(missingStats))
catVals=manifest[(manifest['treatment']=='categorical') & (manifest['class']=='Predictor')].index.values
distinctVals = pd.DataFrame(df[catVals].T.apply(lambda x: x.nunique(), axis=1)).rename(columns={0: 'distinctVals'})

#missingStats=pd.DataFrame(missingStats)#.columns(['missingRt'])

left_merge = pd.DataFrame(missingStats).merge(featureImp, left_index=True,right_index=True, how='left')
#print(left_merge)
left_merge=left_merge.rename(columns={0: 'missingRate', 'featureImpVal': 'featureImpVal'})
left_merge = left_merge.merge(distinctVals, left_index=True,right_index=True, how='left')
left_merge = manifest.merge(left_merge, left_index=True,right_index=True, how='left')
#left_merge.columns(['missingRate','featureImpVal'])


def classTreatmentFx(row):
    if ((row['class'] == 'Predictor') & (row['treatment'] == 'categorical')):
        if row['distinctVals'] > 10:
            return 'group by risk/size'
        else:
            return 'binarize'
    else:
        return 'ignore treatment'
            
    
classTreatment = left_merge.apply(classTreatmentFx, axis=1).astype('category')
classTreatment.cat.set_categories(['binarize','group by risk/size','assign risk score','ignore treatment'],inplace=True)
left_merge["classTreatment"] =classTreatment 
#print(left_merge)

left_merge=left_merge.drop(['types'], axis=1)

print('featureImpVal will be "NaN" if 1) the variable is not a predictor, 2) the missing rate is below the given value (default=0.2) or 3) if the given varience threshold (default=0.001) wasn''t reached')
inputSetup2=qgrid.show_grid(left_merge)
inputSetup2
#print(pd.DataFrame(data=featureImp['featureImpVal'],index=featureImp['vars']))
#Z = pd.concat([pd.DataFrame(missingStats), df[labCol]], axis=1, join='inner')
#varImp[varImp['vars'] in manifest['treatment']=='categorical']]
#varImp['emp_title' in varImp['vars']]
#for index, row in varImp.iterrows():
    #print(row['vars'], row['featureImpVal'])
    #'ID' in row['vars']

featureImpVal will be "NaN" if 1) the variable is not a predictor, 2) the missing rate is below the given value (default=0.2) or 3) if the given varience threshold (default=0.001) wasnt reached


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [21]:
manifest2=inputSetup2.get_changed_df()
pickle.dump(manifest2, open( projName+"_manifestWFImp.p", "wb" ) )

In [22]:

from numpy import loadtxt
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.grid_search import GridSearchCV

#cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
parameters = {'max_depth': [3,5], 'min_child_weight': [1,3,5],'learning_rate': [0.1, 0.05, 0.01]
              , 'n_estimators': [100, 300, 500], 'subsample': [0.7,0.8,0.9]
              , 'colsample_bytree': [0.8, 1]}
# optimized_GBM = GridSearchCV(XGBRegressor(ind_params), 
#                             cv_params, 
#                              scoring = 'neg_mean_absolute_error', cv = 10, n_jobs = -1)
###############
#parameters  = {'max_depth': [5], 'min_child_weight': [1],'learning_rate': [0.1], 'n_estimators': [100], 'subsample': [1]
#              , 'colsample_bytree': [1]
#              ,'gamma':[0],'max_delta_step':[0],'reg_alpha':[0], 'reg_lambda':[1], 'scale_pos_weight':[1],'silent':[1]
#              ,'objective':['reg:linear'],}

optimized_GBM = GridSearchCV(XGBRegressor()
                             ,parameters ,scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1)



###############
optimized_GBM.fit(X_train, y_train)


model = XGBRegressor()
model.fit(X_train, y_train.values.ravel())

y_pred = optimized_GBM.predict(X_test)

rms = sqrt(mean_squared_error(y_test.values.ravel(), y_pred))
print("RMSQ: "+str(round(rms,3)))

C:\Users\family\Anaconda3\envs\LC\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\family\Anaconda3\envs\LC\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


RMSQ: 0.002
